# 논문 토큰 호출용 클래스 정의

In [62]:
import json, base64, requests
from Crypto.Cipher import AES

class AESTestClass:
    def __init__(self, plain_txt, key):
        # iv, block_size 값은 고정
        self.iv = 'jvHJ1EFA0IXBrxxz'
        self.block_size = 16
        self.plain_txt = plain_txt
        self.key = key

    def pad(self):
        # PKCS#7 패딩
        number_of_bytes_to_pad = self.block_size - len(self.plain_txt) % self.block_size
        ascii_str = chr(number_of_bytes_to_pad)
        padding_str = number_of_bytes_to_pad * ascii_str
        return self.plain_txt + padding_str

    def encrypt(self):
        cipher = AES.new(self.key.encode('utf-8'), AES.MODE_CBC, self.iv.encode('utf-8'))
        padded_txt = self.pad()
        encrypted_bytes = cipher.encrypt(padded_txt.encode('utf-8'))
        # URL-safe Base64
        encrypted_str = base64.urlsafe_b64encode(encrypted_bytes).decode('utf-8')
        return encrypted_str

# 토큰 호출 함수

In [63]:
from datetime import datetime
# 나중에 AESTestClass도 호출해야 함.

def call_access_token(MAC_ADDRESS, API_KEY, CLIENT_ID):
    # 맥주소
    mac = (MAC_ADDRESS or "").strip().strip('"').strip("'").upper().replace(":", "-")
    if not mac:
        raise SystemExit("MAC_ADDRESS가 비어있음")

    # datetime 생성
    dt = datetime.now().strftime('%Y%m%d%H%M%S')

    # JSON 페이로드 생성
    payload = {
        "mac_address": mac,
        "datetime": dt
    }
    plain_json = json.dumps(payload, separators=(',', ':'))

    # AES 암호화 → Base64
    aes = AESTestClass(plain_txt=plain_json, key=API_KEY)
    b64_cipher = aes.encrypt()

    # 인코딩 + 토큰 요청
    endpoint = "https://apigateway.kisti.re.kr/tokenrequest.do"
    params = {
        "accounts": b64_cipher,
        "client_id": CLIENT_ID
    }

    response = requests.get(endpoint, params=params, timeout=10)
    response.raise_for_status()

    data = response.json()
    token = data['access_token'] 
    return token

# 논문 API 호출 함수

In [64]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from dotenv import load_dotenv
load_dotenv(override=True)

# 관련 논문 검색 함수
def ARTI_search(client_id, token, query):
    url = "https://apigateway.kisti.re.kr/openapicall.do"
    params = {
        "client_id": client_id,
        "token": token,
        "version": 1.0,
        "action": "search",
        "target": "ARTI",
        "searchQuery": query,
        'curPage': 1, # 현재페이지 번호
        'rowCount': 10, # 디스플레이 건수(기본값 10, 최대값 100)

    }
    print("🔎 호출 URL 미리보기:", requests.Request('GET', url, params=params).prepare().url)
    res = requests.get(url, params=params, timeout=20)
    xml = res.text
    
    return xml


# 단일 논문 조회 함수
def ARTI_browse(client_id, token, cn):
    
    url = "https://apigateway.kisti.re.kr/openapicall.do"
    params = {
        "client_id": client_id,
        "token": token,
        "version": 1.0,
        "action": "browse",
        "target": "ARTI",
        "cn": cn,
        "include": "",
        "exclude": None,
    }
    
    print("🔎 호출 URL 미리보기:", requests.Request('GET', url, params=params).prepare().url)
    res = requests.get(url, params=params, timeout=20)
    xml = res.text
    
    return xml

def xml_to_df(xml):
    # XML 파싱
    root = ET.fromstring(xml)

    # recordList 찾기
    record_list_element = root.find('recordList')

    # 데이터를 담을 리스트
    records = []

    if record_list_element is not None:
        # 각 record에 대해 반복
        for record_element in record_list_element.findall('record'):
            record_data = {}
            # 각 item에 대해 반복
            for item_element in record_element.findall('item'):
                meta_code = item_element.get('metaCode')
                # CDATA 섹션의 텍스트 추출
                value = item_element.text.strip() if item_element.text else ''
                record_data[meta_code] = value
            records.append(record_data)

    df = pd.DataFrame(records)
    return df

# 논문 가져오기

In [ ]:
import os, xmltodict
from dotenv import load_dotenv

load_dotenv(override=True)

CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
ARTI_KEY = os.getenv("SCIENCEON_API_KEY")
MAC_ADDRESS = os.getenv("MAC_ADDRESS")

access_token = call_access_token(MAC_ADDRESS, ARTI_KEY, CLIENT_ID)

cn = 'JAKO200411922932805'

xml = ARTI_browse(CLIENT_ID, access_token, cn)

dict_data = xmltodict.parse(xml)
with open("../data/input_data.json", "w", encoding="utf-8") as f:
    json.dump(dict_data, f, ensure_ascii=False, indent=4)

df = xml_to_df(xml)

print(df['Title'].iloc[0])
print(df['Abstract'].iloc[0])

🔎 호출 URL 미리보기: https://apigateway.kisti.re.kr/openapicall.do?client_id=5e9c4c6079d323f1a6accb980253cac8733c0249be263f11da5fbc9b2f7cb58d&token=7838f041913aeeb709d6861cab05f781c173b5a42764688830cb6130d7b49136&version=1.0&action=browse&target=ARTI&cn=JAKO200411922932805&include=
한국인의 3차원 무릎관절 구축 및 형상 측정
It is necessary to have a model that describes the feature of the knee Joint with a sufficient accuracy. Koreans, however, do not have their own knee joint model to be used in the total knee replacement arthroplasty. They have to use European or American models which do not match Koreans. Three-dimensional visualization techniques are found to be useful in a wide range of medical applications. Three-dimensional imaging studies such as CT(computed tomography) and MRI(magnetic resonance image) provide the primary source of patient-specific data. Three-dimensional knee joint models were constructed by image processing of the CT data of 10 subjects. Using the constructed model, the dimensions 

In [66]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
ARTI_KEY = os.getenv("SCIENCEON_API_KEY")
MAC_ADDRESS = os.getenv("MAC_ADDRESS")

access_token = call_access_token(MAC_ADDRESS, ARTI_KEY, CLIENT_ID)


query = 'Architectural Urbanism'

xml = ARTI_search(CLIENT_ID, access_token, query)
df = xml_to_df(xml)
display(df)

🔎 호출 URL 미리보기: https://apigateway.kisti.re.kr/openapicall.do?client_id=5e9c4c6079d323f1a6accb980253cac8733c0249be263f11da5fbc9b2f7cb58d&token=8c94f717b00188084a2a622ad540c36c7026ac486088ed24f041ad283915f9b6&version=1.0&action=search&target=ARTI&searchQuery=Architectural+Urbanism&curPage=1&rowCount=10


""
